<a href="https://colab.research.google.com/github/Chilik78/Lab_01/blob/main/Lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Подгрузка файлов в Google Colab
from google.colab import files

uploaded = files.upload()

uploaded

In [ ]:
#Просмотр содержимого в папке /content/
import os
# выводим пути к папкам (dirpath) и наименования файлов (filenames) и после этого

for dirpath, _, filenames in os.walk('/content/'):
  for filename in filenames:# во вложенном цикле проходимся по названиям файлов
    print(os.path.join(dirpath, filename))# и соединяем путь до папок и входящие в эти папки файлы с помощью метода path.join()

print("\n")
!ls 
print("\n")
!ls /content/sample_data/

In [ ]:
#Чтение из переменной uploaded
from google.colab import files

uploaded = files.upload()
# посмотрим на тип значений словаря uploaded
type(uploaded['test.csv'])

print("\n")

uploaded_str = uploaded['test.csv'].decode()
print(type(uploaded_str))
print("\n")

print(uploaded_str[:35])

print("\n")
uploaded_list = uploaded_str.split('\r\n')
type(uploaded_list)
print("\n")

# не забудем создать индекс с помощью функции enumerate()

for i, line in enumerate(uploaded_list):
  # начнем выводить записи
  print(line)
  print("\n")
  
  # когда дойдем до четвертой строки
  if i == 3:
    # прервемся
    break



In [ ]:
#Использование функции open() и конструкции with open()
# передадим функции open() адрес файла
# параметр 'r' означает, что мы хотим прочитать (read) файл
f1 = open('/content/train.csv', 'r')

# метод .read() помещает весь файл в одну строку
# выведем первые 142 символа (если параметр не указывать, выведется все содержимое)
print(f1.read(142))

# в конце файл необходимо закрыть
f1.close()

# снова откроем файл
f2 = open('/content/train.csv', 'r')

# пройдемся по нашему объекту в цикле for и параллельно создадим индекс
for i, line in enumerate(f2):
# выведем строки без служебных символов по краям
  print(line.strip())
  # дойдя до четвертой строки, прервемся
  if i == 3:
    break
# не забудем закрыть файл
f2.close()

# скажем Питону: "открой файл и назови его f3"
with open('/content/test.csv', 'r') as f3:
# "пройдись по строкам без служебных символов"
  for i, line in enumerate(f3):
    print(line.strip())
# и "прервись на четвертой строке"
    if i == 3:
      break



In [ ]:
# импортируем библиотеку
import pandas as pd
# применим функцию read_csv() и посмотрим на первые три записи файла train.csv
train = pd.read_csv('/content/train.csv')
train.head(3)

print("\n")

# сделаем то же самое с файлом test.csv
test = pd.read_csv('/content/test.csv')
test.head(3)



In [ ]:
#Обработка и анализ данных. Исследовательский анализ данных (EDA)
train.info()
print("\n")

# для построения графиков воспользуемся новой для нас библиотекой seaborn
import seaborn as sns

# посмотрим насколько значим класс билета длявыживания пассажира
# с помощью x и hue мы можем уместить две категориальные переменные на одном графике
sns.countplot(x = 'Pclass', hue = 'Survived', data = train)
print("\n")

# кто выживал чаще, мужчины или женщины?
sns.countplot(x = 'Sex', hue = 'Survived', data = train)
print("\n")

# выявим пропущенные значения с помощью .isnull() и посчитаем их количество sum()
train.isnull().sum()

In [ ]:


# переменная Cabin (номер каюты), скорее всего, не является самой важной
# избавимся от нее с помощью метода .drop()
# (параметр axis отвечает за столбцы, inplace = True сохраняет изменения)
#train.drop(columns = 'Cabin', axis = 1, inplace = True)

# а вот Age (возраст) точно важен, заменим пустые значения средним арифметическим
train['Age'].fillna(train['Age'].mean(), inplace = True)

# у нас остаются две пустые строки в Embarked, удалим их
train.dropna(inplace = True)

train.isnull().sum()

In [ ]:
#Категориальные переменные
# применим one-hot encoding к переменной Sex (пол) с помощью метода .get_dummies()

pd.get_dummies(train['Sex']).head(3)
sex.head(3)
# удалим первый столбец, он избыточен
sex = pd.get_dummies(train['Sex'], drop_first = True)
#sex.head(3)

embarked = pd.get_dummies(train['Embarked'], drop_first = True)
pclass = pd.get_dummies(train['Pclass'], drop_first = True)

train = pd.concat([train, pclass, sex, embarked], axis = 1)
train.head(3)

In [ ]:
#Отбор признаков

# применим функцию drop() к соответствующим столбцам
train.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Ticket', 'Embarked'], axis = 1, inplace = True)
train.head(3)



In [ ]:
# импортируем класс StandardScaler
from sklearn.preprocessing import StandardScaler
# создадим объект этого класса
scaler = StandardScaler()
# выберем те столбцы, которые мы хотим масштабировать
cols_to_scale = ['Age', 'Fare']
# рассчитаем среднее арифметическое и СКО для масштабирования данных
scaler.fit(train[cols_to_scale])
# применим их
train[cols_to_scale] = scaler.transform(train[cols_to_scale])
# посмотрим на результат
train.head(3)

train.columns = train.columns.map(str)

In [ ]:
#Разделение обучающей выборки на признаки и целевую переменную
# поместим в X_train все кроме столбца Survived
X_train = train.drop('Survived', axis = 1)
# столбец 'Survived' станет нашей целевой переменной (y_train)
y_train = train['Survived']
X_train.head(3)

In [ ]:
#Обучение модели логистической регрессии
# импортируем логистическую регрессию из модуля linear_model библиотеки sklearn
from sklearn.linear_model import LogisticRegression
# создадим объект этого класса и запишем его в переменную model
model = LogisticRegression()
# обучим нашу модель
model.fit(X_train, y_train)

# сделаем предсказание класса на обучающей выборке
y_pred_train = model.predict(X_train)

# построим матрицу ошибок
from sklearn.metrics import confusion_matrix
# передадим ей фактические и прогнозные значения
conf_matrix = confusion_matrix(y_train, y_pred_train)
# преобразуем в датафрейм
conf_matrix_df = pd.DataFrame(conf_matrix)
conf_matrix_df

#Для удобства интерпретации добавим подписи.
conf_matrix_labels = pd.DataFrame(conf_matrix, columns = ['Прогноз погиб', 'Прогноз выжил'], index = ['Факт погиб', 'Факт выжил'])
conf_matrix_labels

1.0

In [ ]:
# рассчитаем метрику accuracy вручную
round((478 + 237)/(478 + 237 + 71 + 103), 3)

0.804

In [ ]:
#Построение прогноза на тестовых данных
test.info()
print("\n")
# для начала дадим датасету привычное название X_test
X_test = test

# заполним пропуски в переменных Age и Fare средним арифметическим
X_test['Age'].fillna(test['Age'].mean(), inplace = True)
X_test['Fare'].fillna(test['Fare'].mean(), inplace = True)

# выполним one-hot encoding категориальных переменных
sex = pd.get_dummies(X_test['Sex'], drop_first = True)
embarked = pd.get_dummies(X_test['Embarked'], drop_first = True)
pclass = pd.get_dummies(X_test['Pclass'], drop_first = True)

# присоединим новые столбцы к исходному датафрейму
X_test = pd.concat([test, pclass, sex, embarked], axis = 1)
# и удалим данные, которые теперь не нужны
X_test.drop(['PassengerId', 'Pclass', 'Name', 'Sex','Cabin', 'Ticket', 'Embarked'], axis = 1, inplace = True)
# посмотрим на результат
X_test.head(3)

# применим среднее арифметическое и СКО обучающей выборки для масштабирования тестовых данных
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
X_test.columns = X_test.columns.map(str)#Остается превратить название столбцов в строки.
y_pred_test = model.predict(X_test) #И сделать прогноз на тестовой выборке.
# посмотрим на первые 10 прогнозных значений
y_pred_test[:10]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: ignored

In [ ]:
#Сохранение результата в новом файле на сервере
# файл с примером можно загрузить не с локального компьютера, а из Интернета
url = 'https://www.dmitrymakarov.ru/wp-content/uploads/2021/11/titanic_example.csv'
# просто поместим его url в функцию read_csv()
example = pd.read_csv(url)
example.head(3)

print("\n")

# возьмем индекс пассажиров из столбца PassengerId тестовой выборки
ids = test['PassengerId']
# создадим датафрейм из словаря, в котором
# первая пара ключа и значения - это id пассажира, вторая - прогноз "на тесте"
result = pd.DataFrame({'PassengerId': ids, 'Survived': y_pred_test})
# посмотрим, что получилось
result.head()

# создадим новый файл result.csv с помощью функции to_csv(), удалив при этом индекс
result.to_csv('result.csv', index = False)
# файл будет сохранен в 'Сессионном хранилище' и, если все пройдет успешно, выведем следующий текст:
print('Файл успешно сохранился в сессионное хранилище!')

NameError: ignored

In [ ]:
#Скачивание обратно на жесткий диск
# применим метод .download() объекта files
files.download('/content/result.csv')